In [32]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from rsome import ro                            # import the ro module
from rsome import grb_solver as grb
from torch.utils.data import DataLoader, TensorDataset

In [3]:
# This is the setup for vehicle pre-allocation problem but it looks like they have some 
# wait and see decisions to make.  I'm not sure how to handle that.  I'm going to try 
# (the last two sentences are from github copilot! Wild!)
I, J = 1, 10
r = np.array([4.50, 4.41, 3.61, 4.49, 4.38, 4.58, 4.53, 4.64, 4.58, 4.32])
c = 3 * np.ones((I, J))
q = 400 * np.ones(I)

In [16]:
# Shortest path problem or minimum cost flow problem
# min_{x} max_{c in U(c)} c^Tx
# Constraints x[i, j]

# Creating the edge list so that one can only travel south or west

edges = np.zeros((25, 25))
for i in range(5):
    for j in range(5):
        if i <4 and j < 4:
            edges[i * 5 + j, i * 5 + j + 1] = 1
            edges[i * 5 + j, (i + 1) * 5 + j] = 1
        elif i== 4 and j < 4:
            edges[i * 5 + j, i * 5 + j + 1] = 1
        elif i < 4 and j == 4:
            edges[i * 5 + j, (i + 1) * 5 + j] = 1
edge_count = int(edges.sum())

In [27]:
# generating the theta such that some of the covariates are independent of cost
np.random.seed(0)
d = 10
Theta = np.random.binomial(1, 0.5, (edge_count, d))
irrelevant_zs = np.random.choice(range(d), size = 2)
Theta[:, irrelevant_zs] = 0

In [28]:
def generate_data(N, d, Theta):
    # Generating zs
    z = np.random.normal(0, 1, (N, d))

    # Computing cost with some noise
    cost_wo_noise = ((1/d)**(0.5) * np.matmul(z, Theta.T) + 3)**5 + 1
    noise = np.random.uniform(low = 0.75, high = 1.25, size = (N, edge_count))
    cost = cost_wo_noise * noise
    return z, cost 

In [30]:
# Data generation for training and calibration
N_train = 1000
z_train, cost_train = generate_data(N_train, d, Theta)
N_calib = 500
z_calib, cost_calib = generate_data(N_calib, d, Theta)

In [34]:
# Creating the dataset for the model
device = ("cuda" if torch.cuda.is_available() else "cpu")
to_tensor = lambda r : torch.tensor(r).to(torch.float32).to(device)
z_train, z_cal = to_tensor(z_train), to_tensor(z_calib)
c_train, c_cal = to_tensor(cost_train), to_tensor(cost_calib)
train_dataset = TensorDataset(z_train, c_train)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

/home/srayan/miniconda3/envs/genIP/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  This is separate from the ipykernel package so we can avoid doing imports until


In [37]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from collections import OrderedDict

class MLP(nn.Module):
    def __init__(self, hidden_sizes):
        super(MLP, self).__init__()
        order_dict = []
        for i in range(len(hidden_sizes) - 1):
            order_dict.append(('Linear Layer {}'.format(i), nn.Linear(hidden_sizes[i], hidden_sizes[i+1])))
            if i < len(hidden_sizes) - 2:
                order_dict.append(('BatchNorm Layer {}'.format(i), nn.BatchNorm1d(hidden_sizes[i+1])))
                order_dict.append(('ReLU Layer {}'.format(i), nn.ReLU()))
        self.mlp = nn.Sequential(OrderedDict(order_dict))
    
    def forward(self, x):
        return self.mlp(x)

In [38]:
model = MLP([d, 64, 128, 64, edge_count]).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 1_000

for epoch in range(num_epochs):
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()  # Zero the gradients
        outputs = model(batch_X)  # Forward pass
        loss = criterion(outputs, batch_y)  # Compute the loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Update weights

    if epoch % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [1/1000], Loss: 301862.2812
Epoch [101/1000], Loss: 129071.9375
Epoch [201/1000], Loss: 71389.2812
Epoch [301/1000], Loss: 74906.1016
Epoch [401/1000], Loss: 43061.4609
Epoch [501/1000], Loss: 17194.7734
Epoch [601/1000], Loss: 20102.0742
Epoch [701/1000], Loss: 22190.7969
Epoch [801/1000], Loss: 32013.7324
Epoch [901/1000], Loss: 10827.7305


In [42]:
def box_score_quantile(alpha, model, z_calib, c_calib):
    model.eval()
    with torch.no_grad():
        losses = np.linalg.norm((model(z_calib) - c_calib).detach().cpu().numpy(), np.inf, axis=1) 
        N_c = z_calib.shape[0]
        return np.quantile(losses, (N_c + 1)*(1 - alpha)/N_c) #, np.sort(losses)

In [ ]:
def box_solve_RO(model, CP_score, edges, z0, edge_count, vertex_count):
    c_pred = model(z0)
    c_pred = c_pred.detach().cpu().numpy()

    model = ro.Model()

    w = model.dvar((vertex_count, vertex_count) 'B')
    c = model.rvar((vertex_count, vertex_count))
    uset = (ro.norm(c[i], np.inf) <= CP_score)

    model.minmax(c @ w, uset)
    model.st(w <= 1)
    model.st(w >= 0)
    model.st(p @ w <= B)

    model.solve(grb)